<a href="https://colab.research.google.com/github/parkmina365/text_mining/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84%EC%98%81%ED%99%94%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install 'jpype1<1'
!pip install konlpy

     |████████████████████████████████| 3.6 MB 5.3 MB/s 
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 86 kB 5.3 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from konlpy.tag import Okt

okt = Okt()
def ok_tokenizer(text):
  tokens_ko = okt.morphs(text)
  return tokens_ko

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
my_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', my_path)
sys.path.insert(0,my_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
upload= files.upload()

In [ ]:
!ls

drive  notebooks  ratings_test.txt  sample_data


In [ ]:
!mv ratings_train.txt /content/drive/MyDrive/cakd3_colab/textmining/dataset
### 리눅스 명령어. moving

In [ ]:
!mv ratings_test.txt /content/drive/MyDrive/cakd3_colab/textmining/dataset

In [ ]:
import pandas as pd

train_df = pd.read_csv('/content/drive/MyDrive/cakd3_colab/textmining/dataset/ratings_train.txt', sep='\t')
test_df = pd.read_csv('/content/drive/MyDrive/cakd3_colab/textmining/dataset/ratings_test.txt', sep='\t')
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [ ]:
train_df['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [ ]:
test_df['label'].value_counts()

1    25173
0    24827
Name: label, dtype: int64

In [ ]:
import re

train_df.info()
train_df.fillna(' ', inplace=True)
train_df['document'] = train_df['document'].apply(lambda x:re.sub(r"\d+", " ", x))
train_df.drop('id', axis=1, inplace=True)
train_df.info()


test_df.fillna(' ', inplace=True)
test_df['document'] = test_df['document'].apply(lambda x: re.sub(r"\d+", " ", x))
test_df.drop('id', axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   document  150000 non-null  object
 1   label     150000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.3+ MB


In [ ]:
from konlpy.tag import Okt

okt = Okt()
def ok_tokenizer(text):
  tokens_ko = okt.morphs(text)
  return tokens_ko

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

tfidf_vect = TfidfVectorizer(tokenizer=ok_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


KeyboardInterrupt: ignored

In [ ]:
# 과제
# GridSearchCV를 이용하여 모델링 및 평가를 아래와 같이 수행하세요
# 로지스틱 리그레션 'C' : [1, 3.5, 4.5, 5.5, 10]
# cv=3, scoring='accuracy', verbose=1
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

tfidf_vect = TfidfVectorizer(tokenizer=ok_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

params = {'C':[1, 3.5, 4.5, 5.5, 10]}
grid = GridSearchCV(LogisticRegression(), param_grid=params, scoring='accuracy', cv=3, verbose=True)
grid.fit(tfidf_matrix_train, train_df['label'])

pred = estimator.predict(X_test)
scores_df = pd.DataFrame(grid.cv_result_)
display(scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score','split1_test_score','split2_test_score' ]])

print(f'Best parameters : {grid.best_params_}')
print(f'Best score : {grid.best_score_}')

tfidf_matrix_test = tfidf_vect.transform(test_df['document'])
preds = grid.best_estimator_.predict(tfidf_matrix_test)
# print(f'Logistic Regression Accuracy : {accuracy_score(test_df['label'], preds)}')

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
 

NameError: ignored